Code is in python 3.10 +. Need to reflect that in environment yaml. 

output_df.to_csv(
        f'../llm_prompt_s_transcript_outputs/{model_name.replace("/", "_")}_{today.strftime("%d_%m_%Y")}.csv',
        index=False
    )

Did not run deekseek. deepseek-ai/DeepSeek-R1 Need to have Agam run it. 

In [1]:
import os,sys
import pandas as pd
import numpy as np
from time import sleep, time
from datetime import date
import threading
from typing import Tuple
from datasets import load_dataset
from openai import OpenAI
import google.generativeai as genai
from google.ai import generativelanguage as glm
import json
from datetime import datetime
import re
import anthropic
today = date.today()

## Configure APIs and Load Data

In [2]:
# If you don't want to use environmental variables you can add your api key directly to api_key
togetherai_api_key = os.environ.get('TOGETHERAI_API_KEY')
openai_api_key = os.environ.get('OPENAI_API_KEY')
google_api_key = os.environ.get('GOOGLE_API_KEY')
anthropic_api_key = os.environ.get('ANTHROPIC_API_KEY')

# together.ai key
client_together = OpenAI(api_key=togetherai_api_key,
                base_url='https://api.together.xyz')

#  OpenAI key
client_openai = OpenAI(api_key=openai_api_key)

# Gemini key (https://ai.google.dev/gemini-api/docs/quickstart?lang=python)
genai.configure(api_key=google_api_key)

# Anthropic API Key
client_anthropic = anthropic.Anthropic(api_key=anthropic_api_key)

def get_safety_settings():
    """ 
    Google API Specific
    Set the block threshold to None for each harm category
    Refer https://ai.google.dev/gemini-api/docs/safety-settings to modify the safety settings
    Gemini model information: https://ai.google.dev/gemini-api/docs/models/gemini#gemini-1.5-pro
    """
    safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
    ]
    return safety_settings

Do not need this line for the segments.

In [3]:
# Load the data

#data_directory = '../process_annotations_pipeline/data_sources/'

#video_transcripts = pd.read_csv(data_directory + 'full_raw_video_transcripts.csv')
#video_metadata = pd.read_csv(data_directory + 'inner_id_to_video_metadata.csv')

#video_transcripts.rename(columns = {'transcript_video_id': 'video_id'},
#                         inplace = True)

# Merging dataframes
#df = video_transcripts.merge(
#    video_metadata[['video_id', 'original_video_title']],  # Selecting only video_id and original_video_title from video_metadata
#    on='video_id',                              # Merging on the video_id column
#    how='left'                                 # Keeping all rows from video_transcripts
#)

# Yash has 288
#len(df)

## Create Prompt

In [4]:
def create_prompt(series, lm_type = "lm", whole = True, price = False):
    """
    Generates a structured prompt based on the provided title, transcript, language model type, and whether we.

    Args:
        series (pd.Series): The pandas series  
        lm_type (str, optional): The type of language model the prompt is intended for (default is "lm").
                                 Options are "lm" or "vlm". vlm has a slightly larger prompt to incorporate facial expression.
        whole (bool, optional): Specifies whether to use the entire transcript (True) or a subset (False).
                                Defaults to True.
        price (bool, optional): Only available for vlm. Can the vlm detect price.

    Returns:
        str: A formatted prompt stored in the variable `prompt`, ready for input into a language model.
    """

    # title
    # transcript
    # 'segment_transcript'

    title = series['video_title']

    if whole == True:
        transcript = series['transcript']
    if whole == False:   
        segment_transcript = series['segment_transcript']
    
    if lm_type == "vlm":
        facial_expression = ("\n           - Facial Expressions: Neutral or doubtful (furrowed brows, pursed lips).",
                             "\n           - Facial Expressions: Moderate enthusiasm (mild smiles, slightly raised eyebrows).",
                             "\n           - Facial Expressions: Enthusiastic, energetic (wide smiles, raised eyebrows).")
    else:
        facial_expression = ("",
                             "",
                             "")

    if whole == True:
        whole_transcript_specific = (" and video title",
                            f"""Inputs:
    - Video Title: {title}
    - Transcript: {transcript}""",
                            "\n           - Consistency: Low conviction if the title makes a bold claim, but the transcript lacks matching conviction.",
                            "\n           - Consistency: Medium conviction if the title makes a bold claim, followed by consistent confidence in the transcript.",
                            "\n           - Consistency: High conviction if the title and transcript are strongly aligned.")
    else:
        whole_transcript_specific = ("",
                            f"""Inputs:
    - Transcript: {segment_transcript}""",
                             "",
                             "",
                            "")
        

    prompt = f"""Analyze the YouTube video transcript{whole_transcript_specific[0]} of influencers discussing the US stock market, focusing on stock recommendations and their conviction.
    
    {whole_transcript_specific[1]}
    
    Instructions:
    1. Does the video contain any stock recommendations:
       - Label this as `Stock Recommendations Present` with either "Yes" or "No".
    
    2. If `Stock Recommendations Present` is "Yes", create a list under the key `Recommendations`. Each recommendation should follow this structure:{{"Action": "Buy | Hold | Don't Buy | Sell | Short Sell | Unclear",
         "Justification": "Brief explanation for the action based on the transcript",
         "Conviction Score": "1 | 2 | 3",
         "Ticker Name": "Ticker name"}}
    
       Details for each field:
       - `Action`: Categorize each stock recommendation as:
         - "Buy": Purchase shares of the stock.
         - "Hold":  Retain the stock if already owned, without necessarily
    buying more.
         - "Don't Buy": Refrain from purchasing the stock.
         - "Sell": Sell shares of the stock currently owned.
         - "Short Sell": Sell shares not currently owned, intending to
    buy them back later at a lower price.
         - "Unclear": When the action is not explicitly stated.
       - `Justification`: Provide a brief explanation for the action based on the transcript.
       - `Conviction Score`: Assign a score based on the following criteria:
         - "1" (Low Conviction):
           - Tone: Hesitant or uncertain language, frequent qualifiers (e.g., “maybe,” “possibly”).{facial_expression[0]}
           - Delivery: Reserved or doubtful language.{whole_transcript_specific[2]}
         - "2" (Moderate Conviction):
           - Tone: Relatively confident language with some qualifiers.{facial_expression[1]}
           - Delivery: Balanced and moderately positive language.{whole_transcript_specific[3]}
         - "3" (High Conviction):
           - Tone: Strong, assertive language without hesitation.{facial_expression[2]}
           - Delivery: Decisive recommendations with no qualifiers.{whole_transcript_specific[4]}
       - `Ticker Name`: Specify the ticker name of the stock being discussed.
    
    3. If `Stock Recommendations Present` is "No", return the following structure:{{"Stock Recommendations Present": "No",
         "Recommendations": []
       }}
    
    
    Output Requirements:
    - Return only valid JSON that can be directly parsed by JSON libraries.
    - Do not include any additional text, comments, formatting indicators (e.g., `json` or backticks), or explanatory content.
    """

    return prompt

In [5]:
def inference_model(prompt: str,
                    model_source: str,
                    model_name: str,
                    temperature: float) -> str:
    """
    Generates a structured prompt based on the provided title, transcript, language model type, and other options.

    Args:
        prompt (str): The input prompt.
        model_source (str): Source of the model (e.g., 'OpenAI', 'Gemini', or other specified sources).
        model_name (str): Name of the model.
        temperature (float): Temperature setting for the model.
        
    Returns:
        str: A string formatted as JSON or output from the model.
    """

    prompt_json = [{"role": "user", "content": prompt}]
    try:
        match model_source:
            # Try to make Max output tokens
            case "OpenAI":
                model_str = model_name
                chat_completion = client_openai.chat.completions.create(
                    model=model_str,
                    messages=prompt_json,
                    temperature=temperature,
                    max_tokens=2048
                )
                prompt_output = chat_completion.choices[0].message.content

            case "Gemini":
                model_str = model_name
                safety_settings = get_safety_settings()
                # Initialize the model once before the iteration over the rows begin.
                # Link below takes about how to deal with json markdown output
                # https://medium.com/google-cloud/how-to-consistently-output-json-with-the-gemini-api-using-controlled-generation-887220525ae0
                model = genai.GenerativeModel(model_name = model_str, \
                                              safety_settings = safety_settings,
                                              generation_config=genai.GenerationConfig(\
                                                  max_output_tokens=2048,\
                                                  response_mime_type="application/json",\
                                                    temperature=temperature)
                                            )
                
                response = model.generate_content(prompt)
                
                if response.text:
                    prompt_output = response.text
                else:
                    print("Prompt:", prompt)
                    if response.prompt_feedback:
                        print("Prompt Feedback: ", response.prompt_feedback)
                    if response.candidates[0].finish_reason != glm.Candidate.FinishReason.STOP:
                        print("Prompt Finish reason: ", response.candidates[0].finish_reason)
                        print("Prompt Safety Warnings: ", response.candidates[0].safety_ratings)
                    prompt_output = ""

            case "Anthropic":
                response = client_anthropic.messages.create(
                    model=model_name, # Takes parsed parameter in function and uses that model for inference
                    temperature = temperature,
                    max_tokens=2048,
                    messages=[
                        {
                            "role": "user", 
                            "content": prompt
                        }
                    ]
                )
            
                prompt_output = response.content[0].text

            case _:
                model_str = model_name
                chat_completion = client_together.chat.completions.create(
                    model=model_str,
                    messages=prompt_json,
                    temperature=temperature,
                    max_tokens=2048
                )
                prompt_output = chat_completion.choices[0].message.content

        # Don't inference too fast
        sleep(0.4)


    except Exception as e:
        print(e)
        sleep(10.0)
        prompt_output = ""

    # Basic cleaning of ```json start
    # https://community.openai.com/t/json-returning-with-json/584818/8
    def clean_json_string(json_string):
        pattern = r'^```json\s*(.*?)\s*```$'
        cleaned_string = re.sub(pattern, r'\1', json_string, flags=re.DOTALL)
        return cleaned_string.strip()

    prompt_output = clean_json_string(prompt_output).strip('```')
    
    # Remove triple backticks for a couple models that output them 
    return prompt_output

In [6]:
"""
prompt_output = inference_model(prompt = prompt,
                model_source =  "Anthropic",
                model_name = "claude-3-5-sonnet-20241022",
                temperature = 0)
"""

'\nprompt_output = inference_model(prompt = prompt,\n                model_source =  "Anthropic",\n                model_name = "claude-3-5-sonnet-20241022",\n                temperature = 0)\n'

In [7]:
#print(prompt_output)

## Experimental Section to Verify Prompt Before Rerunning Models

In [8]:

df = pd.read_csv("../data_needed_for_inference/w_transcripts_for_inference.csv")

for idx, row in df.iterrows():
    # Extract title and transcript
    series = row
    break

In [10]:
#series


In [11]:
prompt = create_prompt(series = series,
              lm_type = 'lm',
              whole = True,
              price = False
             )

In [12]:
print(prompt)

Analyze the YouTube video transcript and video title of influencers discussing the US stock market, focusing on stock recommendations and their conviction.
    
    Inputs:
    - Video Title: 5 Stocks to Buy Now to Double Your Money
    - Transcript:  Hey Bowtie Nation, Joseph Hogue here with the Let's Talk Money channel and finally answering a question I get all the time, what are the stocks to buy that could double fast? And I kind of hate the question but totally understand where it's coming from. Nation, 131 stocks have doubled over the last year with even those mega-cap companies like Apple, Tesla and even NVIDIA finding new highs. And our little Bowtie Nation has benefited from it just as much as anyone with recommendations like Zscaler up 151%, Teladoc up 168% and shares of Fastly jumping by 197%. The problem is, investors think this is normal. Nation, the average annual return on the stock market over the last 30 years is 11%. And that's not bad. An 11% annual return turns a si

# Inference All for Given Model (Whole)

to run for ```whole``` models CHANGE THE PROMPT parameter, output path, and check above prompt

all_lms = {"claude-3-opus-20240229": "Anthropic", 
           "claude-3-5-haiku-20241022": "Anthropic", 
           "claude-3-5-sonnet-20241022": "Anthropic",
           "deepseek-ai/DeepSeek-R1": "Together AI",
           "deepseek-ai/DeepSeek-V3": "Together AI",
           "gemini-1.5-pro-002": "Gemini",
           "gpt-4o-2024-08-06": "OpenAI",
           "gpt-4o-mini-2024-07-18": "OpenAI",
           "gpt-4-turbo-2024-04-09": "OpenAI",
           "meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo": "Together AI",
           "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo": "Together AI",
           "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo": "Together AI",
           "Qwen/Qwen2.5-72B-Instruct-Turbo": "Together AI",
           "Qwen/Qwen2.5-7B-Instruct-Turbo": "Together AI",
           "mistralai/Mistral-7B-Instruct-v0.2": "Together AI",
           "mistralai/Mixtral-8x7B-Instruct-v0.1": "Together AI",
           "mistralai/Mixtral-8x22B-Instruct-v0.1": "Together AI",
          }

DID NOT RUN DEEKSEEK 

"deepseek-ai/DeepSeek-R1": "Together AI",

           "deepseek-ai/DeepSeek-V3": "Together AI",

In [ ]:
## whole
"""
df = pd.read_csv("../data_needed_for_inference/w_transcripts_for_inference.csv")

print(df.columns)



print(df.shape)
"""

In [14]:
df.columns

Index(['video_id', 'video_title', 'transcript'], dtype='object')

In [17]:
df.shape

(288, 3)

In [16]:
all_lms = {"claude-3-opus-20240229": "Anthropic", 
           "claude-3-5-haiku-20241022": "Anthropic", 
           "claude-3-5-sonnet-20241022": "Anthropic",
           "deepseek-ai/DeepSeek-R1": "Together AI",
           "deepseek-ai/DeepSeek-V3": "Together AI",
           "gemini-1.5-pro-002": "Gemini",
           "gpt-4o-2024-08-06": "OpenAI",
           "gpt-4o-mini-2024-07-18": "OpenAI",
           "gpt-4-turbo-2024-04-09": "OpenAI",
           "meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo": "Together AI",
           "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo": "Together AI",
           "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo": "Together AI",
           "Qwen/Qwen2.5-72B-Instruct-Turbo": "Together AI",
           "Qwen/Qwen2.5-7B-Instruct-Turbo": "Together AI",
           "mistralai/Mistral-7B-Instruct-v0.1": "Together AI",
           "mistralai/Mixtral-8x7B-Instruct-v0.1": "Together AI",
           "mistralai/Mixtral-8x22B-Instruct-v0.1": "Together AI",
          }

# Loop through each model
for model_name, model_source in all_lms.items():
    print(model_name)

    # Prepare a DataFrame to store results
    output_df = pd.DataFrame(columns=[
        'video_id',
        'video_title',
        'transcript',
        'model_source',
        'model_name',
        'date_of_inference',
        'prompt_used',
        'prompt_output',
        'video_source'
    ])
    

    # Initialize a counter
    iteration_count = 0
    
    # Loop through each row in the DataFrame
    # video_source will need to be modified 
    for idx, row in df.iterrows():

        # Increment the counter
        iteration_count += 1
    
        # Print progress every 50 iterations
        if iteration_count % 50 == 0:
            print(f"Processed {iteration_count} rows out of {len(df)}")
        
        # pandas series to put into model
        series = row

        prompt = create_prompt(series = series,
                  lm_type = 'lm',
                  whole = True,
                  price = False
                 )

        
        prompt_output = inference_model(prompt = prompt,
                        model_source =  model_source,
                        model_name = model_name,
                        temperature = 0,)
        
        
        # Append results to output DataFrame using pd.concat
        new_row = pd.DataFrame([{
            'video_id': series['video_id'],
            'video_title': series['video_title'],
            'transcript': series['transcript'], 
            'model_source': model_source,
            'model_name': model_name,
            'date_of_inference': datetime.now().strftime("%Y-%m-%d"),
            'prompt_used': prompt,
            'prompt_output': prompt_output,
            'video_source': None
        }])
    
        output_df = pd.concat([output_df, new_row], ignore_index=True)
    
    # Save the output for this model to a separate CSV file
    today = datetime.now()
    output_df.to_csv(
        f'llm_prompt_w_transcript_outputs/{model_name.replace("/", "_")}_{today.strftime("%d_%m_%Y")}.csv',
        index=False
    )

claude-3-opus-20240229
Processed 50 rows out of 288
Processed 100 rows out of 288
Processed 150 rows out of 288
Processed 200 rows out of 288
Processed 250 rows out of 288
claude-3-5-haiku-20241022
Processed 50 rows out of 288
Processed 100 rows out of 288
Processed 150 rows out of 288
Processed 200 rows out of 288
Processed 250 rows out of 288
claude-3-5-sonnet-20241022
Processed 50 rows out of 288
Processed 100 rows out of 288
Processed 150 rows out of 288
Processed 200 rows out of 288
Processed 250 rows out of 288
deepseek-ai/DeepSeek-R1
Processed 50 rows out of 288
Processed 100 rows out of 288
Processed 150 rows out of 288
Processed 200 rows out of 288
Processed 250 rows out of 288
deepseek-ai/DeepSeek-V3
Processed 50 rows out of 288
Processed 100 rows out of 288
Processed 150 rows out of 288
Processed 200 rows out of 288
Processed 250 rows out of 288
gemini-1.5-pro-002
Processed 50 rows out of 288
Processed 100 rows out of 288
Processed 150 rows out of 288
Processed 200 rows out 

For segments, i dont output the ```segment_transcript```

In [ ]:
!ls 

In [ ]:
output_df # concat date of inference to file name when exporting to csv "<date>____.csv"